# CourtRankRL GRPO Training - Chunk-Based, A100 Memory Efficient

## Agents.md Specifikáció (Chunk-Based)

Ez a notebook a CourtRankRL GRPO alapú reranking modell tanítását végzi el **A100 GPU-n** (80GB VRAM, memory-constrained).

### Főbb jellemzők (Chunk-Based megoldás):
- **Model**: Qwen/Qwen3-4B-Instruct-2507 (4-bit) + QLoRA (rank=32, alpha=64)
- **Training**: TRL GRPOTrainer GRPO algoritmussal
  - Loss: "dapo" (eliminates length bias)
  - Reward scaling: "batch" (robust - PPO Lite)
  - Importance sampling: "sequence" (stable - GSPO)
- **Dataset**: 98 query (teljes), 20 chunk/slate, **TELJES chunk szöveg** (~500-800 char)
- **Slate stratégia**: Chunk-level retrieval (nem doc aggregáció!) → legrelevánsabb chunk-ok
- **Baseline**: Slate sorrendje = fusion ranking [0,1,2,...] (BM25+FAISS fusion szerint)
- **Hardware**: Batch size 2, grad accumulation 6, 6 generations/prompt (MEMORY EFFICIENT)
- **Training time**: ~25-35 perc (600 steps, vLLM-mel, csökkentett gen miatt)

### Miért chunk-based?
- ✅ **Releváns kontextus**: BM25+FAISS már kiválasztotta a legrelevánsabb chunk-okat
- ✅ **Teljes szöveg**: A model látja, MIÉRT releváns egy dokumentum
- ✅ **Jobb tanulás**: A model megtanulja értékelni a valódi tartalmat, nem csak metaadatokat


In [ ]:
# Környezet setup és csomagok telepítése
# KRITIKUS: Kompatibilis verziók telepítése (Unsloth + TRL + dependencies)

# 1. Core dependencies ELŐSZÖR (stabil verziók)
%pip install -q --upgrade pip
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install -q transformers datasets huggingface_hub
%pip install -q numpy scipy scikit-learn

# 2. Accelerate - FIXÁLT VERZIÓ az is_fsdp2 kompatibilitás miatt
# CRITICAL: accelerate>=0.30.0 szükséges az is_fsdp2 attribútumhoz, de kompatibilisnek kell lennie Unsloth-tal
%pip install -q "accelerate>=0.30.0" --upgrade

# 3. LoRA és kvantizáció
%pip install -q peft bitsandbytes

# 4. TRL (legfrissebb stabil verzió - GRPO támogatással)
%pip install -q --upgrade trl

# 5. Unsloth telepítése UTOLJÁRA (hogy a TRL-hez igazodjon)
%pip install -q --upgrade unsloth diffusers

# 6. vLLM (opcionális, de ajánlott fast_inference-hez)
%pip install -q vllm

# 7. Egyéb függőségek
%pip install -q --upgrade typing_extensions ranx

print("✅ Csomagok telepítve (kompatibilis verziók)")
print("📦 Telepített verziók ellenőrzése:")
import unsloth, torch, transformers, trl, accelerate
print(f"  - PyTorch: {torch.__version__}")
print(f"  - Transformers: {transformers.__version__}")
print(f"  - Accelerate: {accelerate.__version__} (CRITICAL: >=0.30.0 az is_fsdp2 támogatáshoz)")
print(f"  - TRL: {trl.__version__}")
print(f"  - Unsloth: {unsloth.__version__}")

# FONTOS FIGYELMEZTETÉS a verzióról
if tuple(map(int, accelerate.__version__.split('.')[:2])) < (0, 30):
    print(f"⚠️  FIGYELEM: Accelerate verzió ({accelerate.__version__}) < 0.30.0")
    print(f"   A monkey patch fog futni a training során az is_fsdp2 kompatibilitásért")
    print(f"   Ajánlott: pip install accelerate>=0.30.0 --upgrade && RESTART KERNEL")


In [ ]:
# Importok
import os, warnings
os.environ["TORCHDYNAMO_DISABLE"] = "1"      # hivatalos flag a dynamo kikapcsolására
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"  # ártalmatlan, de segít fallbackben
# (opcionális, ha Unsloth figyeli)
os.environ["UNSLOTH_COMPILE"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore", message=".*An output with one or more elements was resized.*")

import json
import sys
import re
import random
from pathlib import Path
from typing import Dict, List

import numpy as np
import pandas as pd
import torch
from datasets import Dataset
from unsloth import FastLanguageModel
from trl.trainer.grpo_trainer import GRPOTrainer
from trl.trainer.grpo_config import GRPOConfig
from huggingface_hub import login
from sklearn.metrics import ndcg_score
from scipy.stats import entropy as scipy_entropy
from sklearn.model_selection import train_test_split
from ranx import Qrels, Run, evaluate

print("✅ Importok betöltve (Unsloth + TRL + sklearn + scipy + ranx)")
print(f"PyTorch verzió: {torch.__version__}")
print(f"CUDA elérhető: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memória: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


In [ ]:
# Importok
import os, warnings
os.environ["TORCHDYNAMO_DISABLE"] = "1"      # hivatalos flag a dynamo kikapcsolására
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"  # ártalmatlan, de segít fallbackben
# (opcionális, ha Unsloth figyeli)
os.environ["UNSLOTH_COMPILE"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore", message=".*An output with one or more elements was resized.*")

import json
import sys
import re
import random
from pathlib import Path
from typing import Dict, List

import numpy as np
import pandas as pd
import torch
from datasets import Dataset
from unsloth import FastLanguageModel
from trl.trainer.grpo_trainer import GRPOTrainer
from trl.trainer.grpo_config import GRPOConfig
from huggingface_hub import login
from sklearn.metrics import ndcg_score
from scipy.stats import entropy as scipy_entropy
from sklearn.model_selection import train_test_split
from ranx import Qrels, Run, evaluate

print("✅ Importok betöltve (Unsloth + TRL + sklearn + scipy + ranx)")
print(f"PyTorch verzió: {torch.__version__}")
print(f"CUDA elérhető: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memória: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


In [ ]:
# Konfiguráció - MEMORY EFFICIENT (A100 80GB) + CURRICULUM LEARNING
MODEL_NAME = "unsloth/Qwen3-4B-Instruct-2507"
SLATE_SIZE = 20

# Curriculum Learning konfiguráció
CURRICULUM_ENABLED = True  # Engedélyezve - LEGNAGYOBB JAVULÁSI POTENCIÁL
CURRICULUM_STRATEGY = "filtered_dataset"  # "filtered_dataset" vagy "weighted_sampling"
# weighted_sampling: könnyebb példákat gyakrabban választjuk ki
# Ez biztosítja, hogy a modell fokozatosan megtanulja jól rangsorolni
GROUP_SIZE = 6
LORA_RANK = 32  # 64→32: csökkentett rank a memória miatt
LORA_ALPHA = LORA_RANK  # alpha követi a rank-t
LORA_DROPOUT = 0.05
MAX_SEQ_LENGTH = 9216  # 8192→12288: növelt érték a hosszabb promptokhoz (9600 token + margó)
GPU_MEMORY_UTILIZATION = 0.6  # 0.88→0.5 csökkentve a rendelkezésre álló szabad memória miatt
USE_GRADIENT_CHECKPOINTING = "unsloth"

LEARNING_RATE = 5e-5
MAX_STEPS = 300
SAVE_STEPS = 300  # Save more frequently for checkpoint recovery
EVAL_STEPS = 50
LOGGING_STEPS = 10
WARMUP_STEPS = 50
GRADIENT_ACCUMULATION_STEPS = 3  # 3→6: nagyobb accumulation, kisebb batch
NUM_GENERATIONS = 6  # 14→6: csökkentett generációk
PER_DEVICE_BATCH_SIZE = 1  # 4→2: FELÉRE csökkentve
# KRITIKUS: generation_batch_size-nak oszthatónak KELL lennie:
#   1. NUM_GENERATIONS-szel (6)
#   2. PER_DEVICE_BATCH_SIZE-zal (2)
# Legkisebb közös többszörös (LCM): 6
GENERATION_BATCH_SIZE = 6  # 6÷6=1, 6÷2=3 ✓
OPTIMIZER_NAME = "paged_adamw_8bit"
LR_SCHEDULER_TYPE = "linear"  # Enhanced: cosine → linear (more stable final convergence)

NDCG_K = 10
ENTROPY_BONUS = 0.01
# Reward clipping range (updated in reward function)
REWARD_CLIP_MIN = -1.0  # Will be updated in enhanced reward function
REWARD_CLIP_MAX = 2.0   # Enhanced: wider range for positive improvements
TRAIN_SPLIT = 0.8
SEED = 42

BASE_PATH = Path(os.getenv("WORKSPACE_PATH", "/workspace"))

BASELINE_METRICS_PATH = BASE_PATH / "baseline_query_metrics.csv"
SLATE_FILE = BASE_PATH / "training_slates.jsonl"
OUTPUT_DIR = BASE_PATH / "artifacts" / "grpo_policy"
METRICS_FILE = OUTPUT_DIR / "metrics.json"

print("📋 A100 80GB - Memory Efficient Konfiguráció:")
print(f"  Model: {MODEL_NAME}")
print(f"  LoRA Rank: {LORA_RANK} (csökkentve memória miatt)")
print(f"  GPU Memory Utilization: {GPU_MEMORY_UTILIZATION} (0.5 = 50%)")
print(f"  Batch: {PER_DEVICE_BATCH_SIZE} × {GRADIENT_ACCUMULATION_STEPS} = {PER_DEVICE_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS} (effective)")
print(f"  Generations: {NUM_GENERATIONS}, Generation batch: {GENERATION_BATCH_SIZE}")
print(f"  Steps per generation: {GENERATION_BATCH_SIZE // PER_DEVICE_BATCH_SIZE}")
print(f"  Steps: {MAX_STEPS}")
print(f"  ⚠️  Memória optimalizált beállítások aktívak!")

if not SLATE_FILE.exists():
    raise FileNotFoundError(f"❌ Slate fájl nem található: {SLATE_FILE}")

if not BASELINE_METRICS_PATH.exists():
    raise FileNotFoundError(f"❌ Baseline metrikák fájl nem található: {BASELINE_METRICS_PATH}")

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:
# Segédfüggvények
def calculate_ndcg(ranked_indices: List[int], true_relevance: List[float], k: int = 10) -> float:
    if not true_relevance or not ranked_indices or max(true_relevance) == 0:
        return 0.0
    y_true = np.array(true_relevance)
    max_score = len(ranked_indices)
    y_score = np.zeros_like(y_true, dtype=float)
    for i, idx in enumerate(ranked_indices[:k]):
        if idx < len(y_true):
            y_score[idx] = max_score - i
    if np.sum(y_score) == 0:
        return 0.0
    try:
        return float(ndcg_score(y_true.reshape(1, -1), y_score.reshape(1, -1), k=k))
    except:
        return 0.0

def parse_model_ranking(completion: str, slate_size: int = SLATE_SIZE) -> List[int]:
    """
    Megengedőbb parsing - kevésbé szigorú formátum követelmény.
    Ha nincs elég valid index, kiegészítjük a hiányzókat a baseline order szerint.
    """
    try:
        # Próbálunk számokat kinyerni különböző módon
        numbers = []
        
        # 1. Vesszővel elválasztott számok
        parts = completion.split(",")
        for part in parts:
            part = part.strip()
            # Kérjük el minden számot (akár több szám egy cellában)
            for word in part.split():
                word = word.strip().rstrip(".,;")
                if word.isdigit():
                    num = int(word)
                    if 0 <= num < slate_size:
                        numbers.append(num)
        
        # 2. Ha nincs elég, próbálunk szóközzel elválasztott formátumot
        if len(numbers) < slate_size // 3:
            for word in completion.split():
                word = word.strip().rstrip(".,;")
                if word.isdigit():
                    num = int(word)
                    if 0 <= num < slate_size and num not in numbers:
                        numbers.append(num)
        
        # 3. Valid számok gyűjtése
        valid_numbers = []
        seen = set()
        for n in numbers:
            if n not in seen:
                valid_numbers.append(n)
                seen.add(n)
        
        # 4. Ha van elég valid szám (legalább 1/3-a a slate-nek), használjuk
        if len(valid_numbers) >= max(1, slate_size // 3):
            # Kiegészítjük a hiányzó indexeket a baseline order szerint
            missing_indices = [i for i in range(slate_size) if i not in valid_numbers]
            # A hiányzókat a végére teszük (nem zavarjuk meg a modell rankingjét)
            result = valid_numbers + missing_indices
            return result[:slate_size]
        
    except Exception as e:
        pass
    
    # Fallback: baseline order (nem random shuffle - így jobb mint teljesen random)
    return list(range(slate_size))

def create_training_prompt(query_id: str, slate: List[Dict]) -> str:
    """
    Enhanced prompt generation for GRPO training with explicit relevance labels and few-shot examples.
    The prompt uses explicit instructions in ENGLISH to help the model learn proper relevance-based ranking.
    Only the data content (queries, document texts, metadata) remains in HUNGARIAN as it comes from the dataset.
    
    Improvements:
    1. Explicit relevance labels (0/1/2) for each document
    2. Few-shot example ranking output
    3. More detailed relevance criteria
    4. Clearer output format specification
    """
    prompt = f'''# Legal Document Relevance Assessment and Ranking

## TASK
You are helping to rank Hungarian court decision documents by their relevance to a search query.

## SEARCH QUERY
"{query_id}"

## WHAT TO CONSIDER

When ranking documents, think about:
- How well the document topic matches the query
- Whether the content directly addresses what the query is asking
- The legal domain and court type (may or may not be relevant)
- The relevance labels (0=not relevant, 1=relevant, 2=highly relevant) - these can guide you
- BM25 and FAISS scores show some similarity signals, but content is most important

## RANKING INSTRUCTIONS

You need to rank the {len(slate)} documents by their relevance to the search query.

'''
    
    separator_line = "\u2500" * 20
    
    # Enhanced document presentation with EXPLICIT RELEVANCE LABELS
    for idx, doc in enumerate(slate):
        chunk_text = doc.get('text', '')[:800]
        bm25 = doc.get('bm25_score', 0)
        faiss = doc.get('faiss_score', 0)
        relevance = doc.get('relevance', 0)
        court = doc.get('court', 'N/A')
        domain = doc.get('domain', 'N/A')
        year = doc.get('year', 'N/A')
        
        # Relevance label mapping
        rel_label = "NOT RELEVANT" if relevance == 0 else ("RELEVANT" if relevance == 1 else "HIGHLY RELEVANT")
        
        prompt += f'''{separator_line}
DOCUMENT [{idx}]

Relevance: {rel_label} (Grade {relevance}/2)
ID: {doc.get('doc_id', 'N/A')} | Court: {court} | Domain: {domain} | Year: {year}
Scores: BM25={bm25:.2f}, FAISS={faiss:.3f}

Content:
{chunk_text}

'''
    
    # Strict ranking instructions to force tangible outputs
    prompt += f'''{separator_line}


### OUTPUT FORMAT (STRICT):
- Respond with EXACTLY one line in this format → RANKING: <comma-separated indices>
- Use document indices 0 to {len(slate)-1}, each exactly once, without extra spaces
- Do NOT include headings, bullet points, Markdown, or explanations before or after the line

Example:
RANKING: 3,0,5,1,4,2,8,6,7,9

RANKING:
'''
    
    return prompt

print("✅ Segédfüggvények definiálva")
print("  📝 Enhanced learning-to-rank prompt template:")
print("     - English instructions with Hungarian data")
print("     - Explicit ranking criteria")
print("     - Step-by-step guidance")
print("     - Clear do's and don'ts")
print("     - Structured document presentation")



In [ ]:
# Slate adatok betöltése
print(f"📂 Slate adatok betöltése: {SLATE_FILE}")
df_slates = pd.read_json(SLATE_FILE, lines=True, encoding='utf-8')
slates_data = df_slates.to_dict('records')
for slate in slates_data:
    slate["slate"] = slate["slate"][:20]
print(f"✅ Betöltve: {len(slates_data)} slate")

sample = slates_data[0]
print(f"\n📋 Minta slate struktúra:")
print(f"  Query ID: {sample['query_id'][:50]}...")
print(f"  Slate elemek: {len(sample['slate'])}")


## 🎯 Multi-Metrika Reward Setup

A standard nDCG@10 optimalizáció helyett **konvex kombinációt** alkalmazunk:
- **nDCG@10** (50%): Rangsor minőség
- **MRR@5** (35%): Első releváns találat pozíciója (üzleti prioritás!)
- **Recall@20** (15%): Completeness

Ez biztosítja, hogy a model a **valódi üzleti értéket** maximalizálja.

In [ ]:
# Multi-Metrika Reward Helper Functions (inline definiálva - nincs külső import)

from collections import Counter
from typing import Optional, Tuple

def compute_mrr(relevances: List[int], k: Optional[int] = None) -> float:
    """
    Mean Reciprocal Rank számítása.
    
    MRR@k = 1 / rank_first_relevant (ha van releváns a top-k-ban)
           = 0 (ha nincs releváns a top-k-ban)
    """
    if k is not None:
        relevances = relevances[:k]
    
    for i, rel in enumerate(relevances, start=1):
        if rel > 0:
            return 1.0 / i
    
    return 0.0


def compute_recall(relevances: List[int], k: int, total_relevant: Optional[int] = None) -> float:
    """
    Recall@k számítása.
    
    Recall@k = |{relevant docs in top-k}| / |{all relevant docs}|
    """
    if not relevances:
        return 0.0
    
    # Relevantes doc-ok száma top-k-ban
    relevant_in_topk = sum(1 for rel in relevances[:k] if rel > 0)
    
    # Összes releváns (ha nincs megadva, akkor az egész listából számoljuk)
    if total_relevant is None:
        total_relevant = sum(1 for rel in relevances if rel > 0)
    
    if total_relevant == 0:
        return 0.0
    
    return relevant_in_topk / total_relevant


def compute_ndcg_metric(relevances: List[int], k: int) -> float:
    """
    nDCG@k számítása sklearn használatával.
    
    Args:
        relevances: Relevancia értékek ranking sorrendben [rel_1, rel_2, ...]
        k: Top-k truncation
    
    Returns:
        nDCG@k érték [0, 1]
    """
    if not relevances or k <= 0:
        return 0.0
    
    # sklearn ndcg_score: [true_relevances], [predicted_scores]
    # predicted_scores: magasabb score = jobb ranking (fordított sorrend)
    relevances_truncated = relevances[:k]
    scores = list(range(len(relevances_truncated), 0, -1))  # [n, n-1, ..., 1]
    
    try:
        ndcg = ndcg_score(
            y_true=[relevances_truncated],
            y_score=[scores],
            k=k
        )
        return float(ndcg)
    except Exception:
        # Edge case: nincs releváns dokumentum
        return 0.0


def compute_multi_metric_reward(
    predicted_indices: List[int],
    slate: List[Dict],
    baseline_metrics: Dict[str, float],
    weights: Optional[Dict[str, float]] = None,
    clip_range: Tuple[float, float] = (-1.0, 1.0),
    use_sigmoid: bool = True,
    mrr_tiebreak_bonus: float = 0.02,
    diversity_penalty_weight: float = 0.0
) -> Tuple[float, Dict[str, float]]:
    """
    Multi-metrika reward számítása sigmoid-alapú stabilizációval.
    
    Javítások:
    - Sigmoid transzformáció a delta-kra → stabilabb gradiens
    - MRR tie-break bonus: ha nDCG/Recall stagnál de MRR javul
    - Diverzitás büntetés: ugyanazon court/domain túlsúly a top-10-ben
    
    Args:
        predicted_indices: Model által prediktált ranking indexek
        slate: Slate adatok [{"relevance": int, "court": str, "domain": str, ...}, ...]
        baseline_metrics: Baseline metrikák {"ndcg@10": float, "mrr@5": float, ...}
        weights: Metrika súlyok {"ndcg10": 0.6, "mrr5": 0.3, "recall20": 0.1}
        clip_range: Reward clipping tartomány (min, max)
        use_sigmoid: Ha True, sigmoid(delta) transzformáció
        mrr_tiebreak_bonus: Bonus ha csak MRR javul (+ε)
        diversity_penalty_weight: Diverzitás büntetés súlya (0 = kikapcsolva)
    
    Returns:
        (total_reward, components_dict)
    """
    # Default weights (updated: 0.6/0.3/0.1)
    if weights is None:
        weights = {"ndcg10": 0.60, "mrr5": 0.30, "recall20": 0.10}
    
    # Relevancia array a predicted ranking szerint
    relevances = [slate[i]["relevance"] for i in predicted_indices]
    
    # Policy metrikák
    policy_ndcg10 = compute_ndcg_metric(relevances, k=10)
    policy_mrr5 = compute_mrr(relevances, k=5)
    
    # Recall@20: total_relevant a teljes slate-ből (fixált candidate pool!)
    total_relevant = sum(1 for doc in slate if doc.get("relevance", 0) > 0)
    policy_recall20 = compute_recall(relevances, k=20, total_relevant=total_relevant)
    
    # Baseline metrikák
    baseline_ndcg10 = baseline_metrics.get("ndcg@10", 0.0)
    baseline_mrr5 = baseline_metrics.get("mrr@5", 0.0)
    baseline_recall20 = baseline_metrics.get("recall@20", 0.0)
    
    # Delta számítás
    delta_ndcg10 = policy_ndcg10 - baseline_ndcg10
    delta_mrr5 = policy_mrr5 - baseline_mrr5
    delta_recall20 = policy_recall20 - baseline_recall20
    
    # Sigmoid transzformáció (stabilabb reward signal)
    if use_sigmoid:
        # sigmoid(x) = 1 / (1 + exp(-k*x)), k=5 → érzékenység
        def sigmoid_transform(delta, k=5.0):
            return 2.0 / (1.0 + np.exp(-k * delta)) - 1.0  # range: [-1, 1]
        
        component_ndcg = weights["ndcg10"] * sigmoid_transform(delta_ndcg10)
        component_mrr = weights["mrr5"] * sigmoid_transform(delta_mrr5)
        component_recall = weights["recall20"] * sigmoid_transform(delta_recall20)
    else:
        # Lineáris (original)
        component_ndcg = weights["ndcg10"] * delta_ndcg10
        component_mrr = weights["mrr5"] * delta_mrr5
        component_recall = weights["recall20"] * delta_recall20
    
    # Total reward
    reward_raw = component_ndcg + component_mrr + component_recall
    
    # ====== TIE-BREAK BONUS: MRR javulás ======
    # Ha nDCG/Recall nem változik jelentősen, de MRR javul → felhasználói élmény++
    if abs(delta_ndcg10) < 0.01 and abs(delta_recall20) < 0.01 and delta_mrr5 > 0.05:
        reward_raw += mrr_tiebreak_bonus
    
    # ====== DIVERZITÁS BÜNTETÉS ======
    # Ha a top-10 túl homogén (ugyanaz a court/domain dominál) → -ε
    if diversity_penalty_weight > 0:
        top10_indices = predicted_indices[:10]
        top10_courts = [slate[i].get("court", "N/A") for i in top10_indices if i < len(slate)]
        top10_domains = [slate[i].get("domain", "N/A") for i in top10_indices if i < len(slate)]
        
        # Shannon entropy (magas = diverzitás, alacsony = homogén)
        def shannon_entropy(items):
            counts = Counter(items)
            total = len(items)
            if total == 0:
                return 0.0
            probs = [c/total for c in counts.values()]
            return -sum(p * np.log(p + 1e-9) for p in probs)
        
        court_entropy = shannon_entropy(top10_courts)
        domain_entropy = shannon_entropy(top10_domains)
        
        # Normalizálás: max entropy = log(10) ≈ 2.3
        max_entropy = np.log(10)
        diversity_score = (court_entropy + domain_entropy) / (2 * max_entropy)
        
        # Büntetés ha alacsony diverzitás (< 0.5 → homogén)
        if diversity_score < 0.5:
            penalty = diversity_penalty_weight * (0.5 - diversity_score)
            reward_raw -= penalty
    
    # Clipping
    reward_clipped = np.clip(reward_raw, clip_range[0], clip_range[1])
    
    # Komponensek dictionary (részletes tracking)
    components = {
        "reward_total": float(reward_clipped),
        "reward_raw": float(reward_raw),
        "component_ndcg10": float(component_ndcg),
        "component_mrr5": float(component_mrr),
        "component_recall20": float(component_recall),
        "delta_ndcg10": float(delta_ndcg10),
        "delta_mrr5": float(delta_mrr5),
        "delta_recall20": float(delta_recall20),
        "policy_ndcg10": float(policy_ndcg10),
        "policy_mrr5": float(policy_mrr5),
        "policy_recall20": float(policy_recall20),
        "baseline_ndcg10": float(baseline_ndcg10),
        "baseline_mrr5": float(baseline_mrr5),
        "baseline_recall20": float(baseline_recall20),
    }
    
    return float(reward_clipped), components

print("✅ Multi-Metrika Reward Helper Functions definiálva (inline)")
print(f"   🎯 Multi-metric reward: sigmoid-based, 0.6/0.3/0.1 weights")
print(f"   📦 Nincs külső import - minden inline definiálva")

In [ ]:
# Baseline metrikák betöltése (pre-computed from baseline_evaluation.ipynb)
BASELINE_METRICS_PATH = BASE_PATH / "baseline_query_metrics.csv"

if not BASELINE_METRICS_PATH.exists():
    print(f"⚠️  FIGYELEM: Baseline metrics fájl nem található: {BASELINE_METRICS_PATH}")
    print(f"   Előbb futtasd le a baseline_evaluation.ipynb notebookot!")
    print(f"   Fallback: Baseline metrikák inicializálása 0-ra (NEM AJÁNLOTT)")
    baseline_metrics_dict = {}
else:
    baseline_df = pd.read_csv(BASELINE_METRICS_PATH, encoding='utf-8')
    baseline_metrics_dict = {
        row["query_id"]: row.to_dict()
        for _, row in baseline_df.iterrows()
    }
    print(f"✅ Baseline metrikák betöltve: {len(baseline_metrics_dict)} query")
    
    # Statisztikák
    if baseline_metrics_dict:
        sample_query = list(baseline_metrics_dict.keys())[0]
        sample_metrics = baseline_metrics_dict[sample_query]
        print(f"\n📊 Elérhető metrikák (példa query):")
        for key in sorted(sample_metrics.keys()):
            if key != "query_id" and isinstance(sample_metrics[key], (int, float)):
                print(f"  • {key}: {sample_metrics[key]:.4f}")
        
        # Aggregált baseline teljesítmény
        avg_ndcg10 = np.mean([m.get("ndcg@10", 0.0) for m in baseline_metrics_dict.values()])
        avg_mrr5 = np.mean([m.get("mrr@5", 0.0) for m in baseline_metrics_dict.values()])
        avg_recall20 = np.mean([m.get("recall@20", 0.0) for m in baseline_metrics_dict.values()])
        
        print(f"\n🎯 Baseline teljesítmény (átlag):")
        print(f"  • nDCG@10:   {avg_ndcg10:.4f}")
        print(f"  • MRR@5:     {avg_mrr5:.4f}")
        print(f"  • Recall@20: {avg_recall20:.4f}")

In [ ]:
# Eval split exportálása összehasonlító értékeléshez
# CRITICAL: Az eval split query_id-k exportálása determinisztikusan reprodukálható formátumban
# Ez biztosítja, hogy a baseline és GRPO modell ugyanazon eval halmazon legyen kiértékelve

print("\n" + "="*80)
print("📤 Eval Split Exportálása")
print("="*80)

# Query ID-k kinyerése az eval_dataset-ből
eval_query_ids = []

for example in eval_dataset:
    prompt = example["prompt"]
    # Ugyanaz a regex parsing logika, mint a reward function-ben
    match = re.search(r'(?:SEARCH QUERY|Query)[:\s"]*([^"\n]+)', prompt)
    query_id = None
    if match:
        query_id = match.group(1)
    else:
        match2 = re.search(r'##\s*SEARCH\s*QUERY\s*\n\s*"([^"]+)"', prompt)
        if match2:
            query_id = match2.group(1)
    
    if query_id and query_id not in eval_query_ids:
        eval_query_ids.append(query_id)

# Rendezés a determinisztikus kimenetért
eval_query_ids.sort()

print(f"\n📊 Eval split statisztikák:")
print(f"  Eval query-k száma: {len(eval_query_ids)}")
print(f"  Train query-k száma: {len(train_dataset)}")
print(f"  Split arány: {len(eval_query_ids) / (len(eval_query_ids) + len(train_dataset)):.1%}")

# Eval split export JSON formátumban
eval_split_data = {
    "query_ids": eval_query_ids,
    "num_queries": len(eval_query_ids),
    "split_params": {
        "train_split": TRAIN_SPLIT,
        "seed": SEED,
        "shuffle": True
    },
    "created_at": pd.Timestamp.now().isoformat()
}

eval_split_path = OUTPUT_DIR / "eval_split.json"
with open(eval_split_path, 'w', encoding='utf-8') as f:
    json.dump(eval_split_data, f, ensure_ascii=False, indent=2)

print(f"\n✅ Eval split exportálva: {eval_split_path}")
print(f"   Ez a fájl használva lesz a baseline és GRPO összehasonlító notebookban")
print(f"   ugyanazon eval halmazon való kiértékeléshez.")

# Opcionális: CSV export is (egyszerűbb betöltéshez)
eval_split_csv = OUTPUT_DIR / "eval_split.csv"
eval_df = pd.DataFrame({"query_id": eval_query_ids})
eval_df.to_csv(eval_split_csv, index=False, encoding='utf-8')
print(f"✅ Eval split CSV: {eval_split_csv}")

print("\n" + "="*80)


In [ ]:
# Model betöltése - Memory Efficient
print(f"🔄 Model betöltése Unsloth-tal: {MODEL_NAME}")
print(f"  ⚙️  GPU Memory Utilization: {GPU_MEMORY_UTILIZATION}")
print(f"  ⚙️  Max LoRA Rank: {LORA_RANK}")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=True,
    dtype=None,
    fast_inference=True,  # CRITICAL: Enable vLLM for GRPO rollouts (2-3x faster inference)
    max_lora_rank=LORA_RANK,
    gpu_memory_utilization=GPU_MEMORY_UTILIZATION,
    max_model_len=MAX_SEQ_LENGTH - 1,  # CRITICAL: vLLM max_model_len must match max_seq_length for long prompts
)

model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    use_gradient_checkpointing=USE_GRADIENT_CHECKPOINTING,
    use_rslora=True,
    random_state=SEED,
)

# Tokenizer padding fix (tensor shape stability)
# CRITICAL: Csak padding_side állítása, NE állítsuk át a pad_token-et, mert a tokenizer újraállítása
# tenzorméret eltérést okozhat a GRPO trainer vLLM belső logikájával
tokenizer.padding_side = "right"
# Note: pad_token beállítása kihagyva - a GRPO trainer és vLLM automatikusan kezeli
# Az explicit pad_token átállítás ütközhet a modell config-tal és tenzor méret problémákat okozhat

print("✅ Model és tokenizer betöltve (Unsloth + vLLM + RSLoRA)")
print(f"  ✓ fast_inference=True: vLLM engedélyezve GRPO rollouts-hoz (2-3x gyorsabb)")
print(f"  ✓ Padding side: {tokenizer.padding_side}")
print(f"  ✓ Pad token ID: {tokenizer.pad_token_id if hasattr(tokenizer, 'pad_token_id') and tokenizer.pad_token_id else 'N/A (auto-managed)'}")
print(f"  ✓ Note: GRPO trainer automatikusan kezeli a tokenizer beállításokat vLLM-mel")

In [ ]:
# GPU memória monitoring model betöltés után
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1e9
    reserved = torch.cuda.memory_reserved(0) / 1e9
    total = torch.cuda.get_device_properties(0).total_memory / 1e9
    free = total - allocated
    
    print(f"\n📊 GPU Memória Model Betöltés Után:")
    print(f"  Allocated: {allocated:.2f} GB")
    print(f"  Reserved: {reserved:.2f} GB")
    print(f"  Free: {free:.2f} GB")
    print(f"  Free %: {(free/total)*100:.1f}%")
    
    if free < 10:
        print(f"  ⚠️  FIGYELEM: Kevés szabad memória ({free:.2f} GB)!")
        print(f"  💡 Opció: További csökkentés lehet szükséges")


In [ ]:
# ====== MULTI-METRIKA REWARD FUNCTION (Sigmoid-Based) ======
# Konvex kombinációja három fő metrikának sigmoid transzformációval:
# - nDCG@10 (60%): Rangsor minőség (növelt súly)
# - MRR@5 (30%): Első releváns találat (gyakorlati használhatóság)
# - Recall@20 (10%): Completeness
#
# Javítások:
# - Sigmoid(delta) → stabilabb gradiens, nem saturálódik
# - MRR tie-break bonus: ha nDCG/Recall stagnál, de MRR javul → +0.02
# - Diverzitás büntetés: homogén top-10 (court/domain) → -ε

# Reward konfiguráció (config.py alapján)
REWARD_WEIGHTS = {
    "ndcg10": 0.60,    # 50% → 60%
    "mrr5": 0.30,      # 35% → 30%
    "recall20": 0.10,  # 15% → 10%
}
REWARD_CLIP_RANGE = (-1.0, 1.0)
REWARD_USE_SIGMOID = True
REWARD_MRR_TIEBREAK_BONUS = 0.02
REWARD_DIVERSITY_PENALTY = 0.05  # Diverzitás büntetés súlya

# Komponens tracking globális változó (logging célokra)
reward_components_log = []

def multi_metric_reward_function(completions, prompts, **kwargs):
    """
    Multi-metrika GRPO reward függvény sigmoid transzformációval.
    
    A reward három metrika súlyozott kombinációja:
    1. nDCG@10 (60%): Rangsor minőség
    2. MRR@5 (30%): Első releváns találat pozíciója (UX prioritás!)
    3. Recall@20 (10%): Releváns találatok lefedettsége
    
    Speciális funkciók:
    - Sigmoid(delta): stabil gradiens, nem saturálódik
    - MRR tie-break: +bonus ha csak MRR javul (felhasználói élmény++)
    - Diverzitás check: homogén top-10 → penalty
    
    Returns:
        List[float]: Reward értékek minden completion-höz
    """
    global reward_components_log
    rewards = []
    batch_components = []
    
    for completion, prompt in zip(completions, prompts):
        try:
            # 1. Query ID extraction
            match = re.search(r'(?:SEARCH QUERY|Query)[:\s"]*([^"\n]+)', prompt)
            query_id = None
            if match:
                query_id = match.group(1)
            else:
                match2 = re.search(r'##\s*SEARCH\s*QUERY\s*\n\s*"([^"]+)"', prompt)
                if match2:
                    query_id = match2.group(1)
            
            if not query_id or query_id not in slate_lookup:
                # Parsing hiba - enyhe büntetés
                rewards.append(-0.1)
                batch_components.append({"error": "query_id_not_found"})
                continue
            
            # 2. Slate lookup
            slate = slate_lookup[query_id]
            
            # 3. Parse model output
            predicted_indices = parse_model_ranking(completion, len(slate))
            
            # 4. Baseline metrikák lookup
            if query_id not in baseline_metrics_dict:
                # Nincs baseline adat - fallback to zero baseline (nem ideális, de biztonságos)
                baseline_metrics = {
                    "ndcg@10": 0.0,
                    "mrr@5": 0.0,
                    "mrr@10": 0.0,  # Kompatibilitás
                    "recall@20": 0.0,
                }
            else:
                baseline_metrics = baseline_metrics_dict[query_id]

        
            
            # 5. Multi-metric reward számítás (helper function from src.reward_metrics)
            reward, components = compute_multi_metric_reward(
                predicted_indices=predicted_indices,
                slate=slate,
                baseline_metrics=baseline_metrics,
                weights=REWARD_WEIGHTS,
                clip_range=REWARD_CLIP_RANGE,
                use_sigmoid=REWARD_USE_SIGMOID,
                mrr_tiebreak_bonus=REWARD_MRR_TIEBREAK_BONUS,
                diversity_penalty_weight=REWARD_DIVERSITY_PENALTY,
            )
            
            rewards.append(reward)
            batch_components.append(components)
            
        except Exception as e:
            # Exception esetén enyhe büntetés
            rewards.append(-0.1)
            batch_components.append({"error": str(e)[:100]})
    
    # Komponensek mentése logginghoz
    reward_components_log.extend(batch_components)
    
    return rewards

print("✅ Multi-Metrika GRPO Reward function definiálva (Sigmoid-Based v2.0)")
print(f"   📊 Reward súlyok:")
print(f"      • nDCG@10:   {REWARD_WEIGHTS['ndcg10']:.0%} (rangsor minőség)")
print(f"      • MRR@5:     {REWARD_WEIGHTS['mrr5']:.0%} (első releváns találat)")
print(f"      • Recall@20: {REWARD_WEIGHTS['recall20']:.0%} (completeness)")
print(f"   🎯 Sigmoid transzformáció: {REWARD_USE_SIGMOID}")
print(f"   🎁 MRR tie-break bonus: +{REWARD_MRR_TIEBREAK_BONUS}")
print(f"   🌈 Diverzitás büntetés: {REWARD_DIVERSITY_PENALTY}")
print(f"   📏 Reward clipping: {REWARD_CLIP_RANGE}")
print(f"   📈 Komponens tracking: Enabled")

In [ ]:
# GRPO Trainer konfiguráció - MEMORY EFFICIENT + CURRICULUM LEARNING
grpo_config = GRPOConfig(
    output_dir=str(OUTPUT_DIR),
    max_steps=MAX_STEPS,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    num_generations=NUM_GENERATIONS,
    generation_batch_size=GENERATION_BATCH_SIZE,  # Explicit: LCM(6, 2) = 6
    optim=OPTIMIZER_NAME,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    loss_type="dr_grpo",
    scale_rewards="batch",
    use_vllm=True,
    vllm_importance_sampling_correction=False,  # vLLM token logprobs nem egyeznek az alap modellével
    importance_sampling_level="sequence",
    mask_truncated_completions=True,
    epsilon=0.1,  # Enhanced: 0.2 → 0.1 (smaller policy updates, more stable)
    bf16=True,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False, "use_unsloth": True},
    max_grad_norm=1.0,
    logging_steps=LOGGING_STEPS,
    logging_first_step=True,
    eval_steps=EVAL_STEPS,
    save_steps=SAVE_STEPS,
    dataloader_num_workers=1,  # 2→1: csökkentett worker
    seed=SEED,
    max_completion_length=8192,  # CRITICAL: explicit completion length (default 256 túl kevés 30 dokumentumhoz)
)

grpo_config.unsloth_num_chunks = 1

print("\n✅ GRPO Trainer konfiguráció kész (Memory Efficient + Curriculum Learning)")
print(f"  ✓ generation_batch_size={GENERATION_BATCH_SIZE}")
print(f"    - osztható num_generations={NUM_GENERATIONS}-szel: {GENERATION_BATCH_SIZE}÷{NUM_GENERATIONS}={GENERATION_BATCH_SIZE//NUM_GENERATIONS}")
print(f"    - osztható per_device_batch={PER_DEVICE_BATCH_SIZE}-gyel: {GENERATION_BATCH_SIZE}÷{PER_DEVICE_BATCH_SIZE}={GENERATION_BATCH_SIZE//PER_DEVICE_BATCH_SIZE}")
print(f"  ✓ Effective batch size: {PER_DEVICE_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"  ✓ Memória optimalizáció: 50% GPU util, 1 worker, rank 32")
if CURRICULUM_ENABLED:
    print(f"  ✓ Curriculum Learning: {CURRICULUM_STRATEGY} stratégia aktiválva")

## 🧪 Multi-Metrika Reward Validation

Most teszteljük a frissített multi-metrika reward függvényt:
- Sigmoid transzformáció működése
- Komponensek (nDCG/MRR/Recall) számítása
- MRR tie-break bonus
- Diverzitás check
- Baseline metrics betöltés

In [ ]:
# Training indítása
print("\n🚀 GRPO TRAINING INDÍTÁSA\n")
print("="*60)

if CURRICULUM_ENABLED:
    print(f"📚 Curriculum Learning aktív: {CURRICULUM_STRATEGY}")
    if CURRICULUM_STRATEGY == "filtered_dataset":
        print("   Training könnyebb példákkal (easy + medium) - jobb tanulási signal")
    print()

try:
    trainer.train()
    print("\n" + "="*60)
    print("✅ Training sikeresen befejezve!")
except Exception as e:
    print(f"\n❌ Training hiba: {e}")
    import traceback
    traceback.print_exc()
    raise

In [ ]:
# Robusztus tréning log vizualizáció (hibaellenőrzéssel)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json

# Megpróbáljuk a TRL trainer log_history-ját használni; ha nincs, esünk vissza a metrics.json-re
log_history = None
try:
    _ = trainer  # ellenőrzés, hogy létezik-e
    log_history = getattr(trainer.state, "log_history", None)
except NameError:
    log_history = None

if log_history:
    logs = pd.DataFrame(log_history)
else:
    # Fallback: metrics.json betöltése és csak a reward trend megjelenítése
    if 'METRICS_FILE' in globals():
        metrics_path = METRICS_FILE
    else:
        metrics_path = Path("data/models/grpo_policy/metrics.json")

    if Path(metrics_path).exists():
        with open(metrics_path, 'r', encoding='utf-8') as f:
            metrics = json.load(f)
        rewards_trend = metrics.get("training_rewards", {}).get("trend", [])
        if rewards_trend:
            plt.figure(figsize=(8, 4))
            plt.plot(rewards_trend, color="royalblue")
            plt.title("Jutalom trend (metrics.json)")
            plt.xlabel("Lépés")
            plt.ylabel("Jutalom")
            plt.tight_layout()
            plt.show()
        else:
            print("⚠️ Nincs elérhető trainer.state.log_history és a metrics.json nem tartalmaz 'training_rewards.trend'-et.")
    else:
        print("⚠️ Nincs elérhető trainer.state.log_history és a metrics.json fájl sem található.")
    raise SystemExit(0)  # Nincs teljes logs, lépünk ki a cellából a további ábrák nélkül

# Innentől a log_history alapján dolgozunk
if isinstance(logs, pd.DataFrame) and not logs.empty:
    # Csak numerikus oszlopok megtartása
    numeric_cols = [c for c in logs.columns if np.issubdtype(pd.Series(logs[c]).dropna().dtype, np.number)]
    if not numeric_cols:
        print("⚠️ A logokban nincs numerikus oszlop a megjelenítéshez.")
    else:
        logs = logs[numeric_cols].copy()

        def pick_col(df, candidates):
            for c in candidates:
                if c in df.columns:
                    return c
            return None

        # Reward (mean) oszlop kiválasztása több lehetséges kulccsal
        reward_candidates = [
            "rewards/reward_function/mean",  # TRL GRPO gyakori
            "rewards/mean",
            "reward/mean",
            "rewards/reward",
            "reward",
        ]
        reward_col = pick_col(logs, reward_candidates)
        if reward_col is None:
            print("⚠️ Nem található reward átlag oszlop a logokban. Kihagyom az 1. ábrát.")

        # Reward szórás oszlop (opcionális)
        reward_std_candidates = [
            "rewards/reward_function/std",
            "rewards/std",
            "reward/std",
        ]
        reward_std_col = pick_col(logs, reward_std_candidates)

        # Loss és KL oszlopok (opcionális)
        loss_col = pick_col(logs, ["loss", "train/loss"])  
        kl_col = pick_col(logs, ["kl", "train/kl", "kl_divergence"]) 

        # Ábrák létrehozása dinamikusan az elérhető metrikák alapján
        fig, axes = plt.subplots(2, 2, figsize=(13, 8))
        axes = axes.flatten()
        ax_idx = 0

        # 1) Reward trend (simított) + opcionális szórás sáv
        if reward_col is not None:
            reward_smooth = logs[reward_col].rolling(window=10, min_periods=1).mean()
            axes[ax_idx].plot(reward_smooth, label=f"Jutalom (átlag, {reward_col})", color="royalblue")
            if reward_std_col is not None:
                lo = reward_smooth - logs[reward_std_col]
                hi = reward_smooth + logs[reward_std_col]
                axes[ax_idx].fill_between(logs.index, lo, hi, alpha=0.2, color="royalblue", label="± szórás")
            axes[ax_idx].set_title("Jutalom trend")
            axes[ax_idx].set_xlabel("Lépés")
            axes[ax_idx].set_ylabel("Jutalom")
            axes[ax_idx].legend()
            ax_idx += 1

        # 2) Loss görbe
        if loss_col is not None and ax_idx < len(axes):
            axes[ax_idx].plot(logs[loss_col], label="Veszteség (loss)", color="firebrick")
            axes[ax_idx].set_title("Tanítási veszteség")
            axes[ax_idx].set_xlabel("Lépés")
            axes[ax_idx].set_ylabel("Loss")
            axes[ax_idx].legend()
            ax_idx += 1

        # 3) KL görbe
        if kl_col is not None and ax_idx < len(axes):
            axes[ax_idx].plot(logs[kl_col], label="KL divergencia", color="darkorange")
            axes[ax_idx].set_title("KL divergencia alakulása")
            axes[ax_idx].set_xlabel("Lépés")
            axes[ax_idx].set_ylabel("KL")
            axes[ax_idx].legend()
            ax_idx += 1

        # 4) Reward szórás külön (ha van és maradt hely)
        if reward_std_col is not None and ax_idx < len(axes):
            axes[ax_idx].plot(logs[reward_std_col], label="Jutalom szórás", color="seagreen")
            axes[ax_idx].set_title("Jutalom szórás")
            axes[ax_idx].set_xlabel("Lépés")
            axes[ax_idx].set_ylabel("Szórás")
            axes[ax_idx].legend()
            ax_idx += 1

        # Ha kevesebb metrika áll rendelkezésre, a maradék tengelyeket rejtsük el
        for i in range(ax_idx, len(axes)):
            axes[i].axis('off')

        plt.tight_layout()
        plt.show()
else:
    print("⚠️ A logok DataFrame üres.")


⚠️ Nincs elérhető trainer.state.log_history és a metrics.json fájl sem található.


In [ ]:
# OPTIONAL: Prompt Quality Check - Sample Completions
# Futtasd TRAINING ELŐTT, hogy lásd, milyen outputokat generál a modell

print("🔬 PROMPT QUALITY CHECK - Sample Completions Generálása\n")
print("Ez a cella teszteli, hogy a modell milyen outputokat generál az új prompttal.")
print("FONTOS: Ez optional, csak debug célra.\n")

# Egy random query kiválasztása
import random
random.seed(42)
test_slate_idx = random.randint(0, len(slates_data)-1)
test_slate_data = slates_data[test_slate_idx]
test_prompt_text = create_training_prompt(test_slate_data["query_id"], test_slate_data["slate"])

# Generálunk néhány completion-t
print(f"Query: {test_slate_data['query_id'][:80]}...")
print(f"Slate size: {len(test_slate_data['slate'])}")
print("-" * 80)

try:
    # Model inference mode
    model.eval()
    
    # Tokenize prompt
    inputs = tokenizer(test_prompt_text, return_tensors="pt", truncation=False, max_length=MAX_SEQ_LENGTH)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    print("\n🎲 3 Sample Completion Generálása...\n")
    
    for i in range(3):
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=1024,  # Növelve 100-ról 2048-ra, hogy elegendő legyen 30 dokumentum rangsorolásához
                do_sample=True,
                temperature=0.8,
                top_p=0.9,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,  # Stop sequence hozzáadása
            )
        
        completion = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
        parsed = parse_model_ranking(completion, len(test_slate_data["slate"]))
        
        print(f"Completion {i+1}:")
        print(f"  Raw output: {completion[:150]}")
        print(f"  Parsed indices: {parsed[:10]}")  # First 10 indices
        print(f"  Valid? {len(parsed) == len(test_slate_data['slate']) and len(set(parsed)) == len(parsed)}")
        print(f"  Teljes ranking: {parsed}")
        print(completion[-200:])
        print(len(tokenizer(test_prompt_text).input_ids))
    
    # Relevance info
    relevances = [doc.get('relevance', 0) for doc in test_slate_data["slate"]]
    print(f"📊 Ground truth relevance distribution:")
    print(f"  Relevant docs (rel>=1): {sum(1 for r in relevances if r >= 1)}/{len(relevances)}")
    print(f"  Highly relevant (rel=2): {sum(1 for r in relevances if r == 2)}/{len(relevances)}")
    print(f"  Baseline order nDCG@10: {calculate_ndcg(list(range(len(relevances))), relevances, k=10):.4f}")
    
    model.train()
    print("\n✅ Prompt quality check complete!")
    
except Exception as e:
    print(f"❌ Error during quality check: {e}")
    print("Ez nem probléma, a training ettől még futhat.")

print("\n" + "="*80)

In [ ]:
# Artifactumok mentése
print("\n💾 Artifactumok mentése...")
model.save_pretrained_merged(str(OUTPUT_DIR), tokenizer, save_method="lora")
print(f"  ✅ LoRA adapter: {OUTPUT_DIR}")

## 📊 Modell Összehasonlítás

Az értékelés és a baseline vs GRPO összehasonlítás a **`notebooks/model_comparison.ipynb`** notebookban történik.

Ez biztosítja, hogy a baseline és GRPO modell **ugyanazon eval halmazon** legyen kiértékelve, így a metrikák összevethetők.


In [ ]:
# Evaluation (ranx)
print("\n📊 Értékelés futtatása (ranx)...")

import torch
import math
from tqdm.auto import tqdm

def evaluate_policy_ranx(dataset_subset, slate_lookup_dict, dataset_name="", batch_size=4, min_tokens=64, max_tokens=512):
    k_values = [5, 10, 20]
    metrics_to_compute = ["map", "mrr"]
    for k in k_values:
        metrics_to_compute.extend([f"ndcg@{k}", f"precision@{k}", f"recall@{k}"])

    qrels_dict = {}
    baseline_run_dict = {}
    policy_run_dict = {}
    parse_successes = 0
    entries = []

    for example in dataset_subset:
        prompt = example["prompt"]
        match = re.search(r'QUERY:\s*"([^"]+)"', prompt)
        if match:
            query_id = match.group(1)
        else:
            match2 = re.search(r'##\s*SEARCH\s*QUERY\s*\n\s*"([^"]+)"', prompt)
            query_id = match2.group(1) if match2 else None
        if not query_id or query_id not in slate_lookup_dict:
            continue

        slate = slate_lookup_dict[query_id]
        query_qrels = {}
        for doc in slate:
            doc_id = doc.get('doc_id')
            relevance = doc.get('relevance', 0)
            if doc_id is not None:
                query_qrels[doc_id] = relevance
        if not query_qrels:
            continue
        qrels_dict[query_id] = query_qrels

        baseline_docs = {}
        for rank, doc in enumerate(slate):
            doc_id = doc.get('doc_id')
            if doc_id is not None:
                baseline_docs[doc_id] = len(slate) - rank
        baseline_run_dict[query_id] = baseline_docs

        entries.append({
            "prompt": prompt,
            "query_id": query_id,
            "slate": slate,
        })

    if not entries:
        print(f"  {dataset_name}: nincs értékelhető lekérdezés.")
        return {
            "baseline_metrics": {},
            "policy_metrics": {},
            "num_queries": 0,
            "positive_improvement_count": 0,
            "positive_improvement_ratio": 0.0,
            "parse_success_rate": 0.0,
            "parse_success_count": 0,
            "per_query_results": [],
        }

    total_entries = len(entries)
    num_batches = math.ceil(total_entries / batch_size)
    device = model.device

    print(f"  {dataset_name}: {total_entries} lekérdezés feldolgozása batchelt generálással...")
    progress_desc = f"{dataset_name} batch feldolgozás" if dataset_name else "Értékelés batch feldolgozás"
    progress_bar = tqdm(total=num_batches, desc=progress_desc, unit="batch")
    original_padding_side = getattr(tokenizer, "padding_side", "right")
    if getattr(tokenizer, "pad_token", None) is None and tokenizer.eos_token is not None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    try:
        with torch.inference_mode():
            for start in range(0, total_entries, batch_size):
                batch_entries = entries[start:start + batch_size]
                prompts = [entry["prompt"] for entry in batch_entries]
                encodings = tokenizer(prompts, padding=True, return_tensors="pt").to(device)
                input_lengths = encodings.attention_mask.sum(dim=1)

                dynamic_max_new_tokens = 0
                for entry in batch_entries:
                    slate_len = len(entry["slate"])
                    dynamic_max_new_tokens = max(dynamic_max_new_tokens, slate_len * 6)
                dynamic_max_new_tokens = int(max(min_tokens, min(max_tokens, dynamic_max_new_tokens)))

                outputs = model.generate(
                    **encodings,
                    max_new_tokens=dynamic_max_new_tokens,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                    do_sample=False,
                    temperature=0.0,
                    num_beams=1,
                )

                for idx, entry in enumerate(batch_entries):
                    input_len = int(input_lengths[idx].item())
                    completion = tokenizer.decode(outputs[idx][input_len:], skip_special_tokens=True)
                    slate = entry["slate"]
                    query_id = entry["query_id"]

                    predicted_indices = parse_model_ranking(completion, len(slate))
                    is_valid_parse = len(predicted_indices) == len(slate) and len(set(predicted_indices)) == len(slate)
                    if not is_valid_parse:
                        predicted_indices = list(range(len(slate)))
                    else:
                        parse_successes += 1

                    policy_docs = {}
                    for rank, slate_idx in enumerate(predicted_indices):
                        if slate_idx < len(slate):
                            doc_id = slate[slate_idx].get("doc_id")
                            if doc_id is not None:
                                policy_docs[doc_id] = len(slate) - rank
                    policy_run_dict[query_id] = policy_docs

                progress_bar.update(1)
    finally:
        tokenizer.padding_side = original_padding_side
        progress_bar.close()
    qrels_ranx = Qrels(qrels_dict)
    baseline_run_ranx = Run(baseline_run_dict)
    policy_run_ranx = Run(policy_run_dict)

    baseline_metrics = evaluate(qrels_ranx, baseline_run_ranx, metrics_to_compute)
    policy_metrics = evaluate(qrels_ranx, policy_run_ranx, metrics_to_compute)

    per_query_results = []
    for query_id in qrels_dict.keys():
        row = {"query_id": query_id}
        for metric in metrics_to_compute:
            baseline_score = baseline_run_ranx.scores.get(metric, {}).get(query_id, 0.0)
            row[f"baseline_{metric}"] = float(baseline_score)
        for metric in metrics_to_compute:
            policy_score = policy_run_ranx.scores.get(metric, {}).get(query_id, 0.0)
            row[f"policy_{metric}"] = float(policy_score)
        baseline_ndcg10 = baseline_run_ranx.scores.get("ndcg@10", {}).get(query_id, 0.0)
        policy_ndcg10 = policy_run_ranx.scores.get("ndcg@10", {}).get(query_id, 0.0)
        row["improvement_ndcg@10"] = float(policy_ndcg10 - baseline_ndcg10)
        per_query_results.append(row)

    improvements = [row["improvement_ndcg@10"] for row in per_query_results]
    positive_improvements = sum(1 for delta in improvements if delta > 0)
    positive_ratio = positive_improvements / len(improvements) if improvements else 0.0
    parse_success_rate = parse_successes / len(per_query_results) if per_query_results else 0.0

    print(f"  {dataset_name}: vége. Baseline nDCG@10 = {baseline_metrics.get('ndcg@10', 0.0):.4f}, Policy nDCG@10 = {policy_metrics.get('ndcg@10', 0.0):.4f}, Javulás = {policy_metrics.get('ndcg@10', 0.0) - baseline_metrics.get('ndcg@10', 0.0):+.4f}")

    return {
        "baseline_metrics": {k: float(v) for k, v in baseline_metrics.items()},
        "policy_metrics": {k: float(v) for k, v in policy_metrics.items()},
        "num_queries": len(qrels_dict),
        "positive_improvement_count": positive_improvements,
        "positive_improvement_ratio": float(positive_ratio),
        "parse_success_rate": float(parse_success_rate),
        "parse_success_count": parse_successes,
        "per_query_results": per_query_results,
    }

train_eval_results = evaluate_policy_ranx(train_dataset, slate_lookup, "Train halmaz")
eval_eval_results = evaluate_policy_ranx(eval_dataset, slate_lookup, "Eval halmaz")


In [ ]:
# Per-query export
training_rewards = []
for log_entry in trainer.state.log_history:
    if "rewards/mean" in log_entry:
        training_rewards.append(log_entry["rewards/mean"]) 

train_per_query_df = pd.DataFrame(train_eval_results["per_query_results"])
train_per_query_csv = OUTPUT_DIR / "train_per_query_results.csv"
train_per_query_df.to_csv(train_per_query_csv, index=False, encoding='utf-8')
print(f"  ✅ Train per-query results: {train_per_query_csv}")

eval_per_query_df = pd.DataFrame(eval_eval_results["per_query_results"])
eval_per_query_csv = OUTPUT_DIR / "eval_per_query_results.csv"
eval_per_query_df.to_csv(eval_per_query_csv, index=False, encoding='utf-8')
print(f"  ✅ Eval per-query results: {eval_per_query_csv}")

In [ ]:
# Metrics export
final_metrics = {
    "model_name": MODEL_NAME,
    "training_samples": len(train_dataset),
    "eval_samples": len(eval_dataset),
    "slate_size": SLATE_SIZE,
    "group_size": GROUP_SIZE,
    "max_steps": MAX_STEPS,
    "learning_rate": LEARNING_RATE,
    "lora_rank": LORA_RANK,
    "lora_alpha": LORA_ALPHA,
    "final_loss": trainer.state.log_history[-1].get("loss", 0.0) if trainer.state.log_history else 0.0,
    "total_steps": len(trainer.state.log_history) if trainer.state.log_history else 0,
    "training_rewards": {
        "mean": float(np.mean(training_rewards)) if training_rewards else 0.0,
        "std": float(np.std(training_rewards)) if training_rewards else 0.0,
        "min": float(np.min(training_rewards)) if training_rewards else 0.0,
        "max": float(np.max(training_rewards)) if training_rewards else 0.0,
        "trend": training_rewards
    },
    "train_evaluation": {
        "baseline_metrics": train_eval_results["baseline_metrics"],
        "policy_metrics": train_eval_results["policy_metrics"],
        "num_queries": train_eval_results["num_queries"],
        "positive_improvement_count": train_eval_results["positive_improvement_count"],
        "positive_improvement_ratio": train_eval_results["positive_improvement_ratio"],
        "parse_success_rate": train_eval_results["parse_success_rate"],
        "parse_success_count": train_eval_results["parse_success_count"]
    },
    "eval_evaluation": {
        "baseline_metrics": eval_eval_results["baseline_metrics"],
        "policy_metrics": eval_eval_results["policy_metrics"],
        "num_queries": eval_eval_results["num_queries"],
        "positive_improvement_count": eval_eval_results["positive_improvement_count"],
        "positive_improvement_ratio": eval_eval_results["positive_improvement_ratio"],
        "parse_success_rate": eval_eval_results["parse_success_rate"],
        "parse_success_count": eval_eval_results["parse_success_count"]
    },
    "status": "completed"
}

with open(METRICS_FILE, 'w', encoding='utf-8') as f:
    json.dump(final_metrics, f, ensure_ascii=False, indent=2)

print(f"  ✅ Metrics: {METRICS_FILE}")
print("\n✅ Minden artifact sikeresen mentve!")